In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from functools import partial

In [2]:
key_file = open("/Users/tristanbrigham/GithubProjects/AI_Training_Data/LicensePlateProject/training_data.txt", "r")
alphabet = open("/Users/tristanbrigham/GithubProjects/AI_Training_Data/LicensePlateProject/alphabet.txt")
image_path = "/Users/tristanbrigham/GithubProjects/AI_Training_Data/LicensePlateProject/"

key_lines = key_file.readlines()
max_val = int(key_lines[0])
print("The max value is: {}".format(max_val))

The max value is: 3055


In [3]:
key_dict = {}
for count, letter in enumerate(alphabet.readlines()):
    key_dict[letter[0]] = count

print(key_dict)

{'_': 0, 'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'H': 8, 'I': 9, 'J': 10, 'K': 11, 'L': 12, 'M': 13, 'N': 14, 'O': 15, 'P': 16, 'Q': 17, 'R': 18, 'S': 19, 'T': 20, 'U': 21, 'V': 22, 'W': 23, 'X': 24, 'Y': 25, 'Z': 26, '0': 27, '1': 28, '2': 29, '3': 30, '4': 31, '5': 32, '6': 33, '7': 34, '8': 35, '9': 36}


In [4]:
keys = []
letters = []
for number in range(1, max_val):
    letter_image = np.loadtxt(image_path + str(number) + ".txt").reshape(80, 60, 1) / 255.0
    letters.append(letter_image)
    
    array = np.zeros(37, dtype="float32")
    key_char = key_lines[number][0]
    array[key_dict[key_char]] = float(1.0)
    keys.append(array)
    
    if number % 500 == 0:
        print("Completed {}".format(number))

Completed 500
Completed 1000
Completed 1500
Completed 2000
Completed 2500
Completed 3000


In [5]:
letters[0].shape

(80, 60, 1)

In [17]:
DefaultConv2D = partial(keras.layers.Conv2D,
                        kernel_size=3, activation='relu', padding="SAME")

model = keras.models.Sequential([
#     DefaultConv2D(filters=128, kernel_size=11, input_shape=[80, 60], name="First_Conv2D", padding="SAME"),
#     DefaultConv2D(filters=128, kernel_size=11, input_shape=[80, 60], name="Second_Conv2D"),
#     keras.layers.MaxPooling2D(pool_size=2),
#     DefaultConv2D(filters=256, kernel_size=7, name="Third_Conv2D"),
#     DefaultConv2D(filters=256, kernel_size=7, name="Fourth_Conv2D"),
#     keras.layers.MaxPooling2D(pool_size=2),
    
    keras.layers.Flatten(),
    
    keras.layers.Dense(units = 2400, activation='relu', name="Dense_1"),
#     keras.layers.Dense(units = 2400, activation='relu', name="Dense_2"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units = 800, activation='relu', name="Dense_3"),
#     keras.layers.Dense(units = 800, activation='relu', name="Dense_4"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units = 200, activation='relu', name="Dense_5"),
#     keras.layers.Dense(units = 200, activation='relu', name="Dense_6"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units = 37, activation='softmax', name="Dense_FINAL")
])

In [19]:
model.compile(optimizer="adam",
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],)

In [20]:
letters = tf.constant(np.array(letters, dtype="float32").reshape(max_val - 1, 80, 60, ))
keys = tf.constant(np.array(keys, dtype="float32").reshape(max_val - 1, len(key_dict), ))

print(letters.shape)
print(keys.shape)

data = tf.data.Dataset.from_tensor_slices((letters, keys))

(3054, 80, 60)
(3054, 37)


In [23]:
model.fit(letters, keys, batch_size=128, epochs=5)
model.summary()

Epoch 1/5


TypeError: 'NoneType' object is not callable